In [37]:
from dotenv import load_dotenv
import os
import time
import json
import requests

### CREDENTIAL CONFIGURATION

In [11]:
load_dotenv()
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
api_key = os.getenv('AZURE_OPENAI_API_KEY')
api_version = os.getenv('API_VERSION')
assistant_id = os.getenv('ASSISTANT_ID')
thread_id = os.getenv('THREAD_ID')
model = os.getenv('MODEL')

In [6]:
#APIs
message_api = azure_endpoint + "openai/threads/" + thread_id + "/messages"
run_api = azure_endpoint + "openai/threads/" + thread_id + "/runs"

params = {
    "api-version": api_version
}

headers = {
    "api-key" : api_key
}

### APIs EXPLORATION

#### 1. POST /messages API --> ADD MESSAGES TO CHAT

In [30]:
def add_messages(params=params, headers=headers, messages : str = 'hello'):
    body = {
        "role": "user",
        "content": messages
        }
    response = requests.post(message_api, 
                                 params=params, 
                                 headers=headers,
                                json=body)

    if response.status_code == 200:
        # Parse the JSON response (assuming the API returns JSON data)
        data = response.json()
        return data
    else:
        # If the request failed, print the status code and the response text
        print(f"Request failed with status code {response.status_code}")
        return response.text

print(add_messages())

{'id': 'msg_NjkT7uxN9P4tXrWdTJj7EBgd', 'object': 'thread.message', 'created_at': 1723364653, 'assistant_id': None, 'thread_id': 'thread_LGcQrmB7xTWy69fR19WZRrsB', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'hello', 'annotations': []}}], 'attachments': [], 'metadata': {}}


#### 2. POST /runs API --> PUSH MESSAGES TO CHATGPT ENGINE

In [31]:
def runs_thread(params=params, headers=headers, assistant_id=assistant_id, model=model):
    body = {
        "assistant_id": assistant_id,
        "model": model
        }

    response = requests.post(run_api,params=params,headers=headers,json=body)

    if response.status_code == 200:
        # Parse the JSON response (assuming the API returns JSON data)
        data = response.json()
        return data
    else:
        # If the request failed, print the status code and the response text
        print(f"Request failed with status code {response.status_code}")
        return response.text

print(runs_thread())

{'id': 'run_5M4upouk6W9DXr3cLotXmrl5', 'object': 'thread.run', 'created_at': 1723364667, 'assistant_id': 'asst_qccNPjxTUX1AlabUDNxCEbbQ', 'thread_id': 'thread_LGcQrmB7xTWy69fR19WZRrsB', 'status': 'queued', 'started_at': None, 'expires_at': 1723365267, 'cancelled_at': None, 'failed_at': None, 'completed_at': None, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-mini', 'instructions': '### Description: ###\nThe Buzzebees SKU Matching Agent is designed to streamline the process of matching products from purchase orders or invoice documents with the corresponding products in the Buzzebees database. The primary function of this agent is to handle discrepancies in SKU codes by comparing product names, which are predominantly in Thai, and identifying the most similar matches. The agent utilizes natural language processing techniques to analyze and compare the textual content of product names, providing a structured output in JSON format that includes the matched product, its SKU

#### 3. GET /messages API --> REQUEST CHAT HISTORY

In [38]:
def get_chat_history(params=params, headers=headers):
    response = requests.get(message_api, params=params, headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
            # Parse the JSON response (assuming the API returns JSON data)
            data = response.json()

            # Check if the response came from assistant or not
            if data["data"][0]["role"] != 'assistant':
                time.sleep(1)
                get_chat_history()
            else:
                return data
            
    else:
        # If the request failed, print the status code and the response text
        print(f"Request failed with status code {response.status_code}")
        return response.text

print(get_chat_history())

{'object': 'list', 'data': [{'id': 'msg_gfQQSAw53uX8wgJM8q2HceXL', 'object': 'thread.message', 'created_at': 1723364684, 'assistant_id': 'asst_qccNPjxTUX1AlabUDNxCEbbQ', 'thread_id': 'thread_LGcQrmB7xTWy69fR19WZRrsB', 'run_id': 'run_5M4upouk6W9DXr3cLotXmrl5', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Hi again! How can I help you today?', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_NjkT7uxN9P4tXrWdTJj7EBgd', 'object': 'thread.message', 'created_at': 1723364653, 'assistant_id': None, 'thread_id': 'thread_LGcQrmB7xTWy69fR19WZRrsB', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'hello', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_aNDtQe96Sm8NMGGIjgIZwdc5', 'object': 'thread.message', 'created_at': 1723362840, 'assistant_id': 'asst_qccNPjxTUX1AlabUDNxCEbbQ', 'thread_id': 'thread_LGcQrmB7xTWy69fR19WZRrsB', 'run_id': 'run_lF6paVxFDoih6qAPxW55Iyl6', 'role': 'assistant', 'content': [{

**GET ONLY LATEST CHAT**

In [34]:
get_chat_history()["data"][0]["content"][0]['text']['value']

'Hi again! How can I help you today?'

### EXTRACT JSON FROM RESPONSE

In [45]:
full_response = get_chat_history()["data"][6]
full_response

{'id': 'msg_UFz5beDgVaHfKB3CA3LM9sCK',
 'object': 'thread.message',
 'created_at': 1723360395,
 'assistant_id': 'asst_qccNPjxTUX1AlabUDNxCEbbQ',
 'thread_id': 'thread_LGcQrmB7xTWy69fR19WZRrsB',
 'run_id': 'run_PzxczUqnauJxmcZR7IQFLuys',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': {'value': 'Thank you for providing the product details from the Buzzebees database. I will now compare the extracted product names from the invoice against the product names in the Buzzebees database to find the best matches, utilizing natural language processing for similarity analysis.\n\n### Matching Process:\n\n#### Extracted Products from Invoice:\n1. **ทาโร เข้มข้น 50g**\n2. **ทาโร ไลท์รสออริจินัล 20g**\n3. **ทาโร เข้มขึ้น 30g**\n4. **ทาโร สาหร่ายเกาหลี 20g**\n5. **ทาโรอบกรอบ เข้มข้น 180**\n6. **ทาโรอบกรอบ บาร์บีคิว 189**\n7. **ทาโร บาร์บีคิว 30g**\n8. **ตาโซ เข้มข้น 50g**\n9. **ทาโร บาร์บีคิว 50g**\n10. **ทาโร ซูเปอร์แซบ 50g**\n\n#### Product Details from Buzzebees:\n1. **ทาโร ปลาเส้น

In [46]:
full_response['content'][0]['text']['value']

'Thank you for providing the product details from the Buzzebees database. I will now compare the extracted product names from the invoice against the product names in the Buzzebees database to find the best matches, utilizing natural language processing for similarity analysis.\n\n### Matching Process:\n\n#### Extracted Products from Invoice:\n1. **ทาโร เข้มข้น 50g**\n2. **ทาโร ไลท์รสออริจินัล 20g**\n3. **ทาโร เข้มขึ้น 30g**\n4. **ทาโร สาหร่ายเกาหลี 20g**\n5. **ทาโรอบกรอบ เข้มข้น 180**\n6. **ทาโรอบกรอบ บาร์บีคิว 189**\n7. **ทาโร บาร์บีคิว 30g**\n8. **ตาโซ เข้มข้น 50g**\n9. **ทาโร บาร์บีคิว 50g**\n10. **ทาโร ซูเปอร์แซบ 50g**\n\n#### Product Details from Buzzebees:\n1. **ทาโร ปลาเส้น 50 กรัม รสเข้มข้น**\n2. **ทาโร ไลท์ 20 กรัม รสออริจินัล**\n3. **ทาโร ปลาเส้น 30 กรัม รสเข้มข้น**\n4. **ทาโร ปลาเส้น 20 กรัม รสสาหร่ายเกาหลี**\n5. **Taro: ทาโร อบกรอบ 18 กรัม รสเข้มข้น**\n6. **Taro: ทาโร อบกรอบ 18 กรัม รสบาร์บีคิว**\n7. **ทาโร ปลาเส้น 30 กรัม รสบาร์บีคิว**\n8. **ทาโร ปลาเส้น 50 กรัม รสเข้มข้น

In [62]:
response_value = full_response['content'][0]['text']['value']
response_value.split('```')[1]

'json\n[\n    {\n        "vendor_product": "ทาโร เข้มข้น 50g",\n        "bzbs_product": "ทาโร ปลาเส้น 50 กรัม รสเข้มข้น",\n        "bzbs_sku": "1-GDS-TARO0-000000072",\n        "probability": 0.8\n    },\n    {\n        "vendor_product": "ทาโร ไลท์รสออริจินัล 20g",\n        "bzbs_product": "ทาโร ไลท์ 20 กรัม รสออริจินัล",\n        "bzbs_sku": "1-GDS-TARO0-000000068",\n        "probability": 0.85\n    },\n    {\n        "vendor_product": "ทาโร เข้มขึ้น 30g",\n        "bzbs_product": "ทาโร ปลาเส้น 30 กรัม รสเข้มข้น",\n        "bzbs_sku": "1-GDS-TARO0-000000069",\n        "probability": 0.6\n    },\n    {\n        "vendor_product": "ทาโร สาหร่ายเกาหลี 20g",\n        "bzbs_product": "ทาโร ปลาเส้น 20 กรัม รสสาหร่ายเกาหลี",\n        "bzbs_sku": "1-GDS-TARO0-000000067",\n        "probability": 0.7\n    },\n    {\n        "vendor_product": "ทาโรอบกรอบ เข้มข้น 180",\n        "bzbs_product": "Taro: ทาโร อบกรอบ 18 กรัม รสเข้มข้น",\n        "bzbs_sku": "1-GDS-TARO0-000000023",\n        "probabilit

In [63]:
json_str = response_value.split('```')[1].replace('\n', '').replace('json', '')
json_response = json.loads(json_str)
json_response[0:3]

[{'vendor_product': 'ทาโร เข้มข้น 50g',
  'bzbs_product': 'ทาโร ปลาเส้น 50 กรัม รสเข้มข้น',
  'bzbs_sku': '1-GDS-TARO0-000000072',
  'probability': 0.8},
 {'vendor_product': 'ทาโร ไลท์รสออริจินัล 20g',
  'bzbs_product': 'ทาโร ไลท์ 20 กรัม รสออริจินัล',
  'bzbs_sku': '1-GDS-TARO0-000000068',
  'probability': 0.85},
 {'vendor_product': 'ทาโร เข้มขึ้น 30g',
  'bzbs_product': 'ทาโร ปลาเส้น 30 กรัม รสเข้มข้น',
  'bzbs_sku': '1-GDS-TARO0-000000069',
  'probability': 0.6}]

In [65]:
def extract_json(response):
    return json.loads(response.split('```')[1].replace('\n', '').replace('json', ''))

### REQUEST MATCHING DATA

**CREATE PROMPT**

In [29]:
from prompt import GetPrompt

default_prompt = GetPrompt('vendor_data', 'bzbs_data').prompt
print(default_prompt)


                Here is the invoice data that is read from vendor invoice:
                -- VENDOR DATA STARTS HERE --

                vendor_data

                -- VENDOR DATA ENDS HERE --

                And, here is the product data from Buzzebees database:
                -- BUZZEBEES DATA STARTS HERE --

                bzbs_data

                -- BUZZEBEES DATA ENDS HERE --

                Please provide the matching result in JSON format.
                


**MAIN FUNCTION**

In [64]:
def request_match(vendor_data : str, bzbs_data : str, params=params, headers=headers):
    
    prompt = GetPrompt(vendor_data, bzbs_data).prompt
    
    add_messages(messages=prompt)

    runs_thread()

    response = get_chat_history()["data"][0]["content"][0]['text']['value']

    result = extract_json(response)

    return result